# Setup

In [1]:
import sys
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
path_to_root = '/content/drive/My Drive/Colab Notebooks/BatuEl_Dissertation'
sys.path.append(path_to_root)
print("Drive mounted.")

data_path = path_to_root + '/data'

Mounted at /content/drive
Drive mounted.


In [2]:
import torch
import tqdm
from reprshift.learning.algorithms import ERM, GroupDRO, Focal
from reprshift.models.hparams import hparams_f
from reprshift.dataset.datasets import MultiNLI, CivilComments
from reprshift.dataset.dataloaders import InfiniteDataLoader, FastDataLoader

# Data

In [3]:
hparams = hparams_f('Focal')
hparams

{'batch_size': 32,
 'last_layer_dropout': 0.5,
 'optimizer': 'adamw',
 'weight_decay': 0.0001,
 'lr': 1e-05,
 'group_balanced': False,
 'num_training_steps': 30001,
 'num_warmup_steps': 0,
 'gamma': 1}

In [4]:
device = "cuda"
train_weights = None
batch_size = hparams['batch_size']

In [5]:
DATASET = 'CivilComments'  # 'CivilComments' , 'MultiNLI'

if DATASET == 'MultiNLI':
    NUM_CLASSES = 3
    NUM_ATTRIBUTES = 2
    train_dataset = MultiNLI(data_path, 'tr', hparams)
    models_path = path_to_root + '/models/models_mnli'
    print(DATASET)
elif DATASET  == 'CivilComments':
    NUM_CLASSES = 2
    NUM_ATTRIBUTES = 8
    train_dataset = CivilComments(data_path, 'tr', hparams, granularity="fine")
    models_path = path_to_root + '/models/models_civilcomments'
    print(DATASET)
else:
    print('Dataset Not Implemented')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

CivilComments


In [6]:
train_loader = InfiniteDataLoader(  dataset=train_dataset,
                                    weights=train_weights,
                                    batch_size=batch_size,
                                    num_workers=1)
steps_per_epoch = len(train_dataset) / batch_size

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


# Model Definition

In [7]:
### Algorithm 15: Focal ###
class Focal(ERM):
    """Focal loss, https://arxiv.org/abs/1708.02002"""
    def __init__(self, num_classes, num_attributes, hparams):
        super(Focal, self).__init__(num_classes, num_attributes, hparams)

    @staticmethod
    def focal_loss(input_values, gamma):
        p = torch.exp(-input_values)
        loss = (1 - p) ** gamma * input_values
        return loss.mean()

    def _compute_loss(self, i, x, y, a, step):
        return self.focal_loss(self.loss(self.predict(x), y), self.hparams["gamma"])

# Model

In [8]:
algorithm_name = 'Focal'
random_seeds = [2] #[0,1,2]
init_state_dict_path = lambda random_seed : models_path + f'/00_randominit/seed{random_seed}/sd_epoch0.pth'
state_dict_PATH = models_path + '/15_focal/'

# Training

In [9]:
start_step = 1
n_steps = 30001 #hparams['num_training_steps']
checkpoint_freq = 1000
train_losses = {}

for seed in random_seeds:
    print('Training Seed:' , seed)
    algorithm = Focal(num_classes=NUM_CLASSES, num_attributes=NUM_ATTRIBUTES, hparams=hparams)
    algorithm.to(device)
    algorithm.load_state_dict(torch.load(init_state_dict_path(seed)))
    train_losses[seed] = []

    train_loader = InfiniteDataLoader(  dataset=train_dataset,
                                        weights=train_weights,
                                        batch_size=batch_size,
                                        num_workers=1)
    train_minibatches_iterator = iter(train_loader)

    for step in tqdm.tqdm(range(start_step, n_steps)):
        ### Training Step ###
        i, x, y, a = next(train_minibatches_iterator)
        minibatch_device = (i, x.to(device), y.to(device), a.to(device))
        algorithm.train()
        step_vals = algorithm.update(minibatch_device, step)
        train_losses[seed].append(step_vals['loss'])

        ### Evaluation ###
        if (step % checkpoint_freq == 0) or (step == n_steps - 1):
            epoch = int(step / checkpoint_freq)
            algorithm_state_dict = algorithm.state_dict()
            algorithm_state_dict_PATH = state_dict_PATH + f'seed{seed}/sd_epoch{epoch}.pth'
            torch.save(algorithm_state_dict, algorithm_state_dict_PATH)

    loss_PATH = state_dict_PATH + f'losses/Loss_{algorithm_name}_{seed}.pth'
    torch.save(train_losses, loss_PATH)

Training Seed: 2


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 30000/30000 [2:20:47<00:00,  3.55it/s]
